In [19]:
import java.io.File
import util.InputReader


typealias PuzzleInput = String

val exampleInput: PuzzleInput = InputReader.getExample(2023, 5)
val puzzleInput: PuzzleInput = InputReader.getPuzzleInput(2023, 5)

In [20]:
exampleInput

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

In [21]:
fun PuzzleInput.seeds(): List<Long> = lines().first().split("seeds: ")[1].split(" ").map { it.toLong() }

exampleInput.seeds()

[79, 14, 55, 13]

In [22]:
data class SeedMapping(val from: String, val to: String, val sourceRange: LongRange, val offset: Long) {
    val destinationStart = sourceRange.first + offset
}

fun SeedMapping(line: String): SeedMapping {
    val (from, to, sourceBegin, sourceEnd, offset) = line.split(" ")
    return SeedMapping(from, to, sourceBegin.toLong()..sourceEnd.toLong(), offset.toLong())
} 

fun SeedMapping(from: String, to: String, string: String): SeedMapping {
    val (destinationBegin, sourceBegin, length) = string.split(" ").map { it.toLong() }
    val offset = destinationBegin - sourceBegin
    val sourceRange = sourceBegin..(sourceBegin + length)
    return SeedMapping(from, to, sourceRange, offset)
}

fun PuzzleInput.seedMappers() =
    lines().drop(2).joinToString("\n").split("\n\n")
        .flatMap { chunk ->
            val (header, rest) = chunk.split("\n", limit = 2)
            val (from, _, to) = header.split(" ")[0].split("-")
            rest.lines().map { SeedMapping(from, to, it) }.toSet()
        }.groupBy { it.from }


exampleInput.seedMappers()

{seed=[SeedMapping(from=seed, to=soil, sourceRange=98..100, offset=-48), SeedMapping(from=seed, to=soil, sourceRange=50..98, offset=2)], soil=[SeedMapping(from=soil, to=fertilizer, sourceRange=15..52, offset=-15), SeedMapping(from=soil, to=fertilizer, sourceRange=52..54, offset=-15), SeedMapping(from=soil, to=fertilizer, sourceRange=0..15, offset=39)], fertilizer=[SeedMapping(from=fertilizer, to=water, sourceRange=53..61, offset=-4), SeedMapping(from=fertilizer, to=water, sourceRange=11..53, offset=-11), SeedMapping(from=fertilizer, to=water, sourceRange=0..7, offset=42), SeedMapping(from=fertilizer, to=water, sourceRange=7..11, offset=50)], water=[SeedMapping(from=water, to=light, sourceRange=18..25, offset=70), SeedMapping(from=water, to=light, sourceRange=25..95, offset=-7)], light=[SeedMapping(from=light, to=temperature, sourceRange=77..100, offset=-32), SeedMapping(from=light, to=temperature, sourceRange=45..64, offset=36), SeedMapping(from=light, to=temperature, sourceRange=64..7

In [23]:
// make a chain of seed mappers, starting at seed, and then returning the next layer on every iteration
fun PuzzleInput.seedChain(seed: String = "seed"): Sequence<List<SeedMapping>> = sequence {
    var current = seed
    while (true) {
        val next = seedMappers()[current] ?: break
        yield(next)
        current = next.first().to
    }
}

exampleInput.seedChain().toList().map{ it.first().from to it.first().to }

[(seed, soil), (soil, fertilizer), (fertilizer, water), (water, light), (light, temperature), (temperature, humidity), (humidity, location)]

In [24]:
fun SeedMapping.mapSeed(seed: Long): Long {
    if (!sourceRange.contains(seed)) throw IllegalArgumentException("Seed $seed is not in range $sourceRange")
    return seed + offset
}

listOf( 98L, 99L, 100L).map { SeedMapping("a", "b", "50 98 2").mapSeed(it) }

[50, 51, 52]

In [25]:
fun PuzzleInput.partOne() = seeds().minOf { seed ->
    var seedValue = seed
    for (mappers in seedChain()) {
        val mapper = mappers.firstOrNull { it.sourceRange.contains(seedValue) }
        
        seedValue = if (mapper == null) seedValue else mapper.mapSeed(seedValue)
    }
    seedValue
}
exampleInput.partOne()

35

In [26]:
puzzleInput.partOne()

88151870

In [27]:
fun PuzzleInput.seedRanges(): List<LongRange> = trim().lines().first().split("seeds: ")[1]
    .run { split(" ").map { it.toLong() } }
    .chunked(2)
    .map { (start, length) -> start until start + length }

exampleInput.seedRanges()

[79..92, 55..67]

In [28]:
fun LongRange.intersects(other: LongRange): Boolean = this.contains(other.first) || this.contains(other.last)
fun LongRange.contains(other: LongRange): Boolean = this.contains(other.first) && this.contains(other.last)

(1L..10L).intersects(5L..15L)

true

In [29]:
fun Set<LongRange>.removeEmpty() = filter { it.first != it.last }.toSet()
fun Set<LongRange>.splitBy(other: LongRange): Set<LongRange> = flatMap { inputRange ->
    when {
        inputRange == other -> setOf(inputRange)
        inputRange.intersects(other) -> {
            val  otherFirst = other.first.coerceAtLeast(inputRange.first)
            val otherLast = other.last.coerceAtMost(inputRange.last)
            setOf(inputRange.first .. otherFirst, otherFirst..otherLast, otherLast..inputRange.last).removeEmpty()
        }
        else -> setOf(inputRange)
    }
}.toSet()

setOf(10L..20L).splitBy(1L..2L).splitBy(99L..100L).splitBy(8L..10L).splitBy(8L..11L).splitBy(13L..15L).splitBy(19L..21L)

[10..11, 11..13, 13..15, 15..19, 19..20]

In [47]:
fun PuzzleInput.partTwo(): Long {
    // start off with each seed range mapping to itself
    val mappings = seedRanges().map { it to it }.toMap().toMutableMap()
    // map each range of the "seed" mappers to the new range
    val seedMappers = seedMappers()["seed"]!!
    println(mappings.values)
    for (mapper in seedMappers) {
        print("Mapping $mapper: ")
        for ((sourceRange, destinationRange) in mappings.toList()) {
            if (destinationRange.intersects(mapper.sourceRange)) {
                // remove the old range
                mappings.remove(sourceRange)
                val newRanges = setOf(de).splitBy(mapper.)
                
            }
        }
    }
    println(mappings)
    
    return -1L
}

puzzleInput.partTwo()

[104847962..108431793, 1212568077..1327462357, 3890048781..4223500385, 1520059863..1737421852, 310308287..323093896, 3492562455..3785530503, 1901414562..2417565422, 2474299950..2627167097, 3394639029..3454329438, 862612782..1038740978]
Mapping SeedMapping(from=seed, to=soil, sourceRange=2044296880..2440371243, offset=-20855844): 1901414562..2417565422 split by 2044296880..2440371243 into [1901414562..2044296880, 2044296880..2417565422]
Mapping SeedMapping(from=seed, to=soil, sourceRange=3839972576..4294967296, offset=-1420457177): Mapping SeedMapping(from=seed, to=soil, sourceRange=699823315..958743033, offset=-425134898): 862612782..1038740978 split by 699823315..958743033 into [862612782..958743033, 958743033..1038740978]
Mapping SeedMapping(from=seed, to=soil, sourceRange=0..431744151, offset=533608135): Mapping SeedMapping(from=seed, to=soil, sourceRange=431744151..592869475, offset=533608135): Mapping SeedMapping(from=seed, to=soil, sourceRange=2936663910..3839972576, offset=45499

-1